
#قیمت گذاری اختیار با روش مونت کارلو


<div dir="rtl">
هدف این نوت بوک بررسی روش های مختلف برای ارزیابی اختیار ها در چارچوب مدل قیمت گذاری بلک شولز با استفاده از پایتون است. به طور خاص، برای قیمت‌گذاری اختیارهای تماس اروپایی به روش‌های مونت کارلو تکیه می‌کنیم و نتایج را با نتایج به‌دست‌آمده از راه‌حل دقیق بلک شولز مقایسه می‌کنیم.

به عنوان یک مقدمه ، مدل بلک شولز بیان می کند که قیمت دارایی به عنوان تابعی از زمان$S(t)$ از یک حرکت هندسی براونی پیروی می کند:

$$dS=S\mu dt + S \sigma dW $$

سپس یک اختیار $f$ داشته باشید که تابعی از $S$ و $t$ خواهد بود. سپس، روندی که این اختیار دنبال می‌کند به شرح زیر خواهد بود:
$$df = \left( \frac{\partial f}{\partial S}\mu S + \frac{\partial f}{\partial t} + \frac{1}{2}\sigma^{2}S^{2}\frac{\partial^{2}f}{\partial S^{2}}\right)dt + \frac{\partial f}{\partial S}\sigma S dz\,$$

اجازه دهید پورتفولیوی زیر $\Pi$ را در نظر بگیریم که شامل گزینه $f$ و $\Delta$ مقدار معینی از $S$ اساسی است:

$$\Pi = f + \Delta S\,$$

سپس، این نمونه کارها روند زیر را دنبال می کند:

$$d\Pi = - df +\frac{\partial f}{\partial S} dS = \left( -\frac{\partial f}{\partial t} - \frac{1}{2}\sigma^{2}S^{2}\frac{\partial^{2} f}{\partial S^{2}} \right)dt\,$$

ما تنظیم کرده ایم:

$$\Delta = \partial f / \partial S\,$$

با ترکیب این معادلات برای $d\Pi$، به این نتیجه می رسیم:

$$\frac{\partial f}{ \partial t} + r\,S\frac{\partial f}{\partial S} +\frac{1}{2}\sigma^{2}S^{2}\frac{\partial^{2}f}{\partial S^{2}} = r\,f\,$$

که معادله بلک شولز است.

حل معادله بلک شولز برای یک اختیار خرید اروپایی $C(S,t)$، با توجه به سررسید $T$ و قیمت اعتصاب $X$ به صورت زیر ارائه می شود:

$$C(S,t) = S\, N(d_{1}) - X\,e^{-r(T-t)}N(d_{2})\,$$

$$d_{1} = \frac{\ln(S/X) + \left(r + \frac{\sigma^{2}}{2}\right)\left(T-t\right)}{\sigma\sqrt{T-t}}\,,$$

$$d_{2} = \frac{\ln(S/X) + \left(r - \frac{\sigma^{2}}{2}\right)\left(T-t\right)}{\sigma\sqrt{T-t}}\,,$$

$$N(d)= \frac{1}{\sqrt{2\pi}}\int_{-d}^{\infty} dx\; e^{-x^{2}/2}\,.$$
<div>

#راه حل دقیق
<div dir="rtl">
اجازه دهید ابتدا روی راه حل دقیق برای قیمت گزینه تماس $C(S,t)$ تمرکز کنیم.

<div>

In [ ]:
import numpy as np
import pandas as pd
from scipy import log,exp,sqrt,stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
plt.style.use('fivethirtyeight')
import numpy.random as npr

<div dir="rtl">
ما راه حل دقیق را مستقیماً تعریف می کنیم:
<div>

In [ ]:
def BS_Call_Exact(S,X,r,sigma,T,t):

    d1 = (log(S/X)+(r+((sigma**2)/2))*(T-t))/(sigma*sqrt((T-t)))
    d2 = d1-sigma*sqrt((T-t))
    
    ST = S*stats.norm.cdf(d1) - X*exp(-r*(T-t))*stats.norm.cdf(d2)
    
    return ST

In [ ]:
t = np.arange(0.75, 1.0, 0.0001)

plt.figure(figsize = (10,8))
sns.lineplot(t,BS_Call_Exact(100,95,0.06,0.3,1,t), label = 'X = 95')
sns.lineplot(t,BS_Call_Exact(100,98,0.06,0.3,1,t), label = 'X = 98')
sns.lineplot(t,BS_Call_Exact(100,100,0.06,0.3,1,t), label='X = 100')
sns.lineplot(t,BS_Call_Exact(100,105,0.06,0.3,1,t), label = 'X = 105')

plt.xlabel('t')
plt.ylabel('C')
plt.ylim(-0.5, 8)

plt.legend()
plt.show()

<div dir="rtl">
به طور خاص، برای $T-t=0$، ما تابع استاندارد برای اختیار خرید در سررسید داریم:
<div>

In [ ]:
s = np.arange(95, 105, 0.01)

plt.figure(figsize = (10,8))
sns.lineplot(s,BS_Call_Exact(s,100,0.06,0.3,1,0.99999), label = 'X = 100')

plt.xlabel('S')
plt.ylabel('C(t=T)')

plt.legend()
plt.show()

#شبیه سازی مونت کارلو

<div dir="rtl">
یک راه جایگزین برای محاسبه قیمت اختیار خرید، استفاده از روش‌های مونت کارلو است. نقطه شروع، گرفتن معادله حرکت براونی برای سهام اصلی $S(t)$ است:

$$dS = S\,r dt +S\sigma dz\,,$$

جایی که ما $\mu =r$ را تنظیم کرده ایم، زیرا در یک محیط بدون ریسک کار می کنیم. این معادله را می توان با طرح اویلر گسسته کرد. با انجام این کار، سطح شاخص سهام $S_T$ را در $T$ آینده با سطح $S_t$ به صورت زیر بدست می آوریم:

$$S_{T} = S_{t}\exp\left(\left(r-\frac{1}{2}\sigma^{2}\right)(T-t) +\sigma \sqrt{T-t}\,z\right)\,,$$

که در آن $z$ یک متغیر تصادفی است که از توزیع نرمال پیروی می کند.
<div>

In [ ]:
r=0.06
sigma = 0.3
T = 1.0
S0 = 100

I = 100000
M = 100

dt = T/M

S = np.zeros((M+1, I))
S[0] = S0
for t in range(1,M+1):
    S[t] = S[t-1]*exp((r -0.5*sigma**2)*dt +sigma*sqrt(dt)*npr.standard_normal(I))

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(S[-1], bins=100)
plt.xlabel('S(T)')
plt.ylabel('frequency')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(S[:, :6])
plt.xlabel('t')
plt.ylabel('S(t)')
plt.show()

<div dir="rtl">
اکنون که در مورد یک اختیار خرید، تابع پرداخت در سررسید $T$ توسط:

$$\ h(S_{T}) = \max \left( S_{T}-X,0\right)\,.$$

سپس، در بازار بدون ریسک، می توان نشان داد که قیمت اختیار خرید به صورت زیر داده می شود:
$$C(T)  = e^{-r(T-t)}\int_{-\infty}^{\infty}dS_{T}\,h(S_{T})\,q(S_{T})\,,$$

که در آن $q(S)$ چگالی احتمال برای فرآیند مارتینگل معادل S_t$ است. این انتگرال در واقع می تواند گسسته شود، و به دست می آید:

$$C(T) = e^{-r(T-t)}\frac{1}{I}\sum_{i = 1}^{I}h(S_{T}^{i})\,,$$

که در آن مجموع بر روی یک مجموعه $S_{T}^{i}$ به دنبال یک حرکت براونی هندسی که می تواند توسط شبیه سازی مونت کارلو ایجاد شود، گرفته می شود.
<div>

In [ ]:
def BS_Call_MC(S,X,r,sigma,T,t,I):
    
    data = np.zeros((I, 2))
    
    z = np.random.normal(0, 1, [1, I])
    
    #z = npr.standard_normal(I)
    
    ST = S*exp((T-t)*(r - 0.5*sigma**2)+sigma*sqrt(T-t)*z)
    
    data[:,1] = ST - X
    
    average = np.sum(np.amax(data, axis=1))/float(I)
    
    return np.exp(-r*(T-t))*average

<div dir="rtl">

ما نتیجه مونت کارلو را برای مقادیر مختلف مقایسه می کنیم و می بینیم که نتیجه خوب یداده :
<div>

In [ ]:
print('Monte Carlo:',BS_Call_MC(100,95,0.06,0.3,1,.999,100000))
print('Exact: ',BS_Call_Exact(100,95,0.06,0.3,1,.999))

In [ ]:
print('Monte Carlo:',BS_Call_MC(100,95,0.06,0.3,1,.75,100000))
print('Exact: ',BS_Call_Exact(100,95,0.06,0.3,1,.75))

In [ ]:
df = pd.DataFrame(columns=['Iter','BSc'])

for i in range(1, 100000, 500):
    
    df = df.append({'Iter': i, 'BSc': BS_Call_MC(100,95,0.06,0.3,1,0.75,i)}, ignore_index=True)


plt.figure(figsize=(10,8))

plt.hlines(BS_Call_Exact(100,95,0.06,0.3,1,0.75),xmin=0,xmax= 100000,linestyle='dotted',colors='red',label = 'Exact')
plt.plot(df.set_index('Iter'),lw = 1.5, label = 'Monte Carlo')

plt.title('S=100, X=95, T-t=0.25')
plt.xlabel('Iterations')
plt.ylabel('C')
plt.ylim(BS_Call_Exact(100,95,0.06,0.3,1,0.75)-1,BS_Call_Exact(100,95,0.06,0.3,1,0.75)+1)
plt.legend()
plt.show()

In [ ]:
df = pd.DataFrame(columns=['Iter','BSc'])

for i in range(1, 100000, 500):
    
    df = df.append({'Iter': i, 'BSc': BS_Call_MC(100,95,0.06,0.3,1,0.99,i)}, ignore_index=True)


plt.figure(figsize=(10,8))

plt.plot(df.set_index('Iter'),lw = 1.5)
plt.hlines(BS_Call_Exact(100,95,0.06,0.3,1,0.99),xmin=0,xmax= 100000,linestyle='dotted',colors='red')

plt.title('S=100, X=95, T-t=0.001')
plt.xlabel('Iterations')
plt.ylabel('C')
plt.ylim(BS_Call_Exact(100,95,0.06,0.3,1,0.99)-1,BS_Call_Exact(100,95,0.06,0.3,1,0.99)+1)

plt.show()
